In [1]:
%reset

import matplotlib 
import pandas as pd
import datetime
import dateutil.parser
import numpy as np
import plotly.express as px

#import plotly.figure_factory as ff

import matplotlib.pyplot as pl

import pdb

%matplotlib notebook

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [3]:
files = ['./data/us-counties-2020.txt','./data/us-counties-2021.txt','./data/us-counties-2022.txt']

In [4]:
frames = []
for fi in files:
    df = pd.read_csv(fi, dtype={"fips":"string"}).fillna('-1')
    frames.append(df)
df = pd.concat(frames)    
df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [5]:
#Get days from start date
dates = df['date'].tolist()
days_from_start = \
    [(dateutil.parser.isoparse(dates[i]) - dateutil.parser.isoparse(dates[0])).days for i in range(len(dates))]
days_from_start = np.array(days_from_start)

In [6]:
#Convert data to numpy arrays
values = np.array(df['cases'].tolist())
fips = np.array(df['fips'].tolist())
#FIPS values for every county
county_fips = np.unique(fips)

In [7]:
num_days = 200

county_total = np.zeros(len(county_fips))
county_delta = np.zeros(len(county_fips))

for dayi in range(0,num_days):
    if (dayi % 20 == 0):
        print("day = {}".format(dayi))
    day_match = np.where(days_from_start == dayi)[0]
    this_day_vals = values[day_match]
    this_day_fips = fips[day_match]
    
    county_indices = np.searchsorted(county_fips, this_day_fips)
    county_delta[county_indices] = this_day_vals - county_total[county_indices]
    county_total[county_indices] = this_day_vals
        
    #make plot
    fig = px.choropleth({'fips':county_fips, 'values': np.log10(county_delta)}, \
                            geojson=counties, locations='fips', color='values',
                           color_continuous_scale="Viridis", labels = {'values':'Log10(Cases)'},
                           range_color=(0.0, 4),
                           scope="usa",
                          title = 'Days since Jan. 21, 2020: {}'.format(dayi)
                          )
    fig.update_layout(coloraxis_colorbar_x=0.)
    fig.write_image('./plots/all_counties_delta_log10_{}.png'.format(dayi))    

day = 0


<ipython-input-7-2b11480d626c>:18: RuntimeWarning: divide by zero encountered in log10
  fig = px.choropleth({'fips':county_fips, 'values': np.log10(county_delta)}, \
<ipython-input-7-2b11480d626c>:18: RuntimeWarning:

divide by zero encountered in log10



day = 20
day = 40


<ipython-input-7-2b11480d626c>:18: RuntimeWarning:

invalid value encountered in log10



day = 60
day = 80
day = 100
day = 120
day = 140
day = 160
day = 180


In [8]:
num_days_for_gif = 200
filenames = []
for i in range(0,num_days_for_gif):
    filenames.append('./plots/all_counties_delta_log10_{}.png'.format(i))
import imageio
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./plots/covid_cases.gif', images)

<img src="./plots/covid_cases.gif?1" width="750" align="center">
